In [1]:
from __future__ import print_function

import pandas as pd
import os 
import boto3
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d")
    
root_folder = os.getcwd()
root_folder = "/tmp"
print(root_folder)
image_root = '/home/paperspace/data/images/niihau'

/tmp


In [ ]:
# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

In [2]:
import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'hawaii-marine-debris'


In [3]:
import boto3
import pandas as pd

test_annotations_key = "ml/data/keras_annotations_test.csv"
local_test_annotation_path = "{}/keras_annotations_test.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(test_annotations_key, local_test_annotation_path)

# obj = s3_client.get_object(Bucket=bucket_name, Key=test_annotations_key)
test_annotation_df = pd.read_csv(local_test_annotation_path, encoding='utf-8',names=['image_path','x0','y0','x1','y1','label'])
test_annotation_df[:5]

,image_path,x0,y0,x1,y1,label
0,/home/paperspace/data/images/niihau/1_1920_04_...,474,617,524,667,P
1,/home/paperspace/data/images/niihau/1_1920_08_...,511,357,561,407,B
2,/home/paperspace/data/images/niihau/1_1920_08_...,549,767,599,817,C
3,/home/paperspace/data/images/niihau/1_1920_08_...,158,631,208,681,B
4,/home/paperspace/data/images/niihau/1_1920_09_...,256,540,306,590,T


In [4]:
test_annotation_df['s3_key'] = test_annotation_df['image_path'].apply(lambda s: s.replace(image_root,'niihau/jpg'))
test_annotation_df['label'] = test_annotation_df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'wood','V':'Boat'})
test_annotation_df.groupby('label').size().reset_index(name='counts')


In [7]:
test_annotation_df = test_annotation_df.sort_values('s3_key')
test_annotation_df = test_annotation_df.drop(['image_path'], axis=1)

In [10]:
first_row = test_annotation_df.iloc[0]
local_image_path = "{}/{}".format(root_folder, os.path.basename(first_row['s3_key']))
s3_resource.Bucket(bucket_name).download_file(first_row['s3_key'], local_image_path)


x0                                873
y0                                300
x1                                923
y1                                350
label                         Plastic
s3_key    niihau/jpg/1_1008_03_07.jpg
Name: 561, dtype: object

In [33]:
# show images inline
%matplotlib inline


# import keras
import keras
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using TensorFlow backend.


In [154]:

def display_objects(image_file , actual=[], threshold = 0.1):
    image = read_image_bgr(image_file)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(40, 40))

    # preprocess image for network
    image = preprocess_image(image)
    # image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    print("detected {} labels".format(len(labels[0])))
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < threshold :
            print("not enough confidence")
            break
        
        color = label_color(label)
        print(box)
        b = box.astype(int)
        draw_box(draw, b, color=color)
        if score >= threshold:
            caption = "{} {:.3f}".format(label_lookup[label], score)
            print(caption)
        else:
            caption = ""
        draw_caption(draw, b, caption)
    for obj in actual:
        draw_box(draw, obj, color=[31  , 0   , 255])
    plt.figure(figsize=(120, 120))

    plt.imshow(draw)
    plt.show()
    
def detect_objects(image_file, threshold = 0.1):
    image = read_image_bgr(image_file)
    detected_objects = []
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(40, 40))

    # preprocess image for network
    image = preprocess_image(image)
    # image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < threshold:
#             print("not enough confidence")
            break
        
        color = label_color(label)
#         print(box, label, score)
        b = box.astype(int)
        detected_objects.append({'x1':b[0], 'y1':b[1], 'x2': b[2], 'y2':b[3],'label':label,'score':score})

    return detected_objects
    
def detect_debris(row):
    local_image_path = "{}/{}".format(root_folder, os.path.basename(first_row['s3_key']))
    if not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
    display_objects(local_image_path)

In [124]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

#bb1: predicted
#bb2: actual
def overlapped_type(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    
    if x_left==bb1['x1'] and y_top==bb1['y1'] and x_right==bb1['x2'] and y_bottom==bb1['y2']:
        return 'outside'
    elif x_left==bb2['x1'] and y_top==bb2['y1'] and x_right==bb2['x2'] and y_bottom==bb2['y2']:
        return 'within'
    else:
        return 'overwrapped'

    

In [ ]:
model_path = "/tmp/model.h5"
model_key = "ml/model/debris_model_v2.h5"
if not os.path.isfile(model_path):
    s3_resource.Bucket(bucket_name).download_file(model_key, model_path)

model = models.load_model(model_path, backbone_name='resnet50')


In [160]:
test_annotation_df['label_found'] = None
test_annotation_df['wrong_label'] = None
test_annotation_df['detected_bounding'] = None


# deteobjs = detect_objects(local_image_path)
num_found = 0
processed = 0
wrong_label = 0
detection_per_image = {}
for i, row in test_annotation_df[:5].iterrows():
    x1, y1, x2, y2, label = row['x0'], row['y0'], row['x1'], row['y1'], row['label'] #42,347,849,1134
    annotation ={'x1':x1, 'x2':x2, 'y1':y1, 'y2':y2, 'label':label}

    local_image_path = "{}/{}".format(root_folder, os.path.basename(row['s3_key']))

    #download, if it's not there yet.
    if not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
        print("image ready",end='\r')

    if not local_image_path in detection_per_image:
        deteobjs = detect_objects(local_image_path)
        detection_per_image[local_image_path] = deteobjs
        print("Detected objects",end='\r')
    else:
        deteobjs = detection_per_image[local_image_path]
    found = False
    for obj in deteobjs:
        overlap_ratio =get_iou(obj,annotation)
        if(overlap_ratio):
            found = True
            num_found +=1
            test_annotation_df.loc[index, 'detected_bounding'] = overlapped_type(obj,annotation)
            if annotation['label']!=label_lookup[obj['label']][0]:
                test_annotation_df.loc[index, 'wrong_label'] = True
            break
    processed +=1
    message =("%s debris found out of %s" % (num_found, processed))
    print(message,end='\r')
    

image ready
Detected objects
image readyest found 0
Detected objects
image readyest found 0
Detected objects
image readyest found 0
Detected objects
image readyest found 0
Detected objects
image readyest found 1
Detected objects
image readyest found 1
Detected objects
image readyest found 1
Detected objects
image readyest found 1
Detected objects
image readyest found 1


KeyboardInterrupt: 

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

...

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

<Figure size 2880x2880 with 0 Axes>

In [ ]:
test_annotation_df.to_csv("evaluation_result.csv", encoding='utf-8')
s3_client.upload_file('evaluation_result.csv', bucket_name, 'ml/result/evaluation_result_{}.csv'.format(ts))
